In [4]:

#pip install cv2
#pip install mediapipe


#pip install comtypes


SyntaxError: invalid syntax (251544518.py, line 6)

In [1]:
pip install pycaw

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import mediapipe as mp
import math
import time
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [6]:

class HandDetector:

    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, minTrackCon=0.5):

        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.minTrackCon = minTrackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detectionCon, self.minTrackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]
        self.fingers = []
        self.lmList = []

    def findHands(self, img, draw=True):

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw=True):

        xList = []
        yList = []
        bbox = []
        bboxInfo = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                px, py = int(lm.x * w), int(lm.y * h)
                xList.append(px)
                yList.append(py)
                self.lmList.append([px, py])
                if draw:
                    cv2.circle(img, (px, py), 5, (255, 0, 255), cv2.FILLED)
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            boxW, boxH = xmax - xmin, ymax - ymin
            bbox = xmin, ymin, boxW, boxH
            cx, cy = bbox[0] + (bbox[2] // 2), \
                     bbox[1] + (bbox[3] // 2)
            bboxInfo = {"id": id, "bbox": bbox, "center": (cx, cy)}

            if draw:
                cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20), (bbox[0] + bbox[2] + 20, bbox[1] + bbox[3] + 20),
                              (0, 255, 0), 2)

        return self.lmList, bboxInfo

    def fingersUp(self):

        if self.results.multi_hand_landmarks:
            myHandType = self.handType()
            fingers = []
            # Thumb
            if myHandType == "Right":
                if self.lmList[self.tipIds[0]][0] > self.lmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            else:
                if self.lmList[self.tipIds[0]][0] < self.lmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            # 4 Fingers
            for id in range(1, 5):
                if self.lmList[self.tipIds[id]][1] < self.lmList[self.tipIds[id] - 2][1]:
                    fingers.append(1)
                else:
                    fingers.append(0)
        return fingers

    def findDistance(self, p1, p2, img, draw=True):

        if self.results.multi_hand_landmarks:
            x1, y1 = self.lmList[p1][0], self.lmList[p1][1]
            x2, y2 = self.lmList[p2][0], self.lmList[p2][1]
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            if draw:
                cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
                cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

            length = math.hypot(x2 - x1, y2 - y1)
            return length, img, [x1, y1, x2, y2, cx, cy]

    def handType(self):

        if self.results.multi_hand_landmarks:
            if self.lmList[17][0] < self.lmList[5][0]:
                return "Right"
            else:
                return "Left"


In [7]:




if __name__ == "__main__":

    cap = cv2.VideoCapture(0)
    cap.set(3, 1280)
    cap.set(4, 720)
    pTime = 0
    detector = HandDetector(detectionCon=0)
    colorR = (255, 0, 255)
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    # volume.GetMute()
    # volume.GetMasterVolumeLevel()
    volRange = volume.GetVolumeRange()
    minVol = volRange[0]
    maxVol = volRange[1]
    vol = 0
    volBar = 400
    volPer = 0

    cx, cy, w, h = 100, 100, 200, 200

    while True:
        success, img = cap.read()
        img = cv2.flip(img, 1)
        img = detector.findHands(img)
        lmList, _ = detector.findPosition(img)

        if lmList:

            length, _, _ = detector.findDistance(4, 8, img)
            cursor = lmList[8]

            vol = np.interp(length, [50, 300], [minVol, maxVol])
            volBar = np.interp(length, [50, 300], [400, 150])
            volPer = np.interp(length, [50, 300], [0, 100])
            print(int(length), vol)
            volume.SetMasterVolumeLevel(vol, None)
            if length < 50:
                cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

                cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
                cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
                cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX,
                            1, (255, 0, 0), 3)

            cTime = time.time()
            fps = 1 / (cTime - pTime)
            pTime = cTime

        cv2.imshow("hands", img)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break



109 -49.76492836875996
140 -41.55086633674203
176 -32.32768197055972
169 -34.138522341298405
170 -33.69724578010904
161 -36.02122234501097
164 -35.3292212544385
164 -35.3292212544385
163 -35.5525342856445
164 -35.3292212544385
162 -35.77541809729342
165 -35.145719818307704
167 -34.563221414923575
145 -40.324724227466106
142 -41.09124828753322
140 -41.64831354494039
137 -42.38806922483838
149 -39.20484842080798
146 -39.99341491335987
138 -42.104654000824915
140 -41.637163612180785
137 -42.52776237079934
139 -41.86735413121907
144 -40.57669566435092
144 -40.480208395603235
149 -39.354112078423476
148 -39.61384782638625
148 -39.484640192315624
148 -39.671118253306894
144 -40.51895403777179
144 -40.50633481653307
147 -39.705521301616216
148 -39.579526229654675
149 -39.2222767167789
153 -38.189131423017024
151 -38.77163832132679
150 -38.96166301176421
150 -38.96166301176421
146 -40.10828817662135
146 -40.053925260230955
136 -42.78769124655509
136 -42.596382914332054
120 -46.978912518879554


78 -57.873519025539416
42 -65.25
58 -63.00531092176111
57 -63.359032260258374
60 -62.42398979592165
68 -60.51174418331016
87 -55.55852616913319
68 -60.552
53 -64.44485734465357
43 -65.25
55 -63.82214390871356
64 -61.365142161801295
72 -59.390554503106124
66 -60.875439833384604
76 -58.243716570610594
99 -52.271873962960754
80 -57.38577180482148
88 -55.10034437755595
80 -57.39391629214118
97 -52.80736694650786
98 -52.65152057138669
91 -54.29225581192602
88 -55.25501538295154
88 -55.21219315309486
86 -55.701258065989606
84 -56.2073096930229
85 -56.07665079246605
86 -55.701258065989606
84 -56.2706104487664
87 -55.50915479847226
84 -56.14111537103007
87 -55.498695826773414
85 -55.93152340904728
84 -56.13343118567963
86 -55.822156308933906
83 -56.42421372841652
87 -55.38396218802212
84 -56.330992625974204
86 -55.63654174667953
85 -56.072053333697234
86 -55.74953958341426
85 -56.000150224721246
82 -56.64486287275003
84 -56.2073096930229
84 -56.2073096930229
84 -56.2073096930229
87 -55.5675143

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
